In [44]:
import pandas as pd
import numpy as np
import yfinance
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
from datetime import date
import matplotlib
matplotlib.use('TkAgg')
plt.rcParams['figure.figsize'] = [12, 7]
plt.rc('font', size=14)

In [43]:
stocksymbols = 'LT.NS'
ticker = yfinance.Ticker(stocksymbols)
end = date.today()
start = "2023-01-01"
df = ticker.history(interval="1d",start=start,end=end)

In [45]:
df['Date'] = pd.to_datetime(df.index)
df['Date'] = df['Date'].apply(mpl_dates.date2num)
df.index = pd.to_datetime(df.index, errors='coerce')
df.index = df.index.strftime('%d-%m-%y')
df = df.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]

In [46]:
df

,Date,Open,High,Low,Close
Date,,,,,
02-01-23,19358.770833,2092.899902,2097.600098,2075.050049,2089.449951
03-01-23,19359.770833,2088.000000,2096.800049,2067.949951,2088.949951
04-01-23,19360.770833,2090.949951,2101.449951,2058.000000,2070.550049
05-01-23,19361.770833,2080.000000,2099.199951,2068.500000,2087.350098
06-01-23,19362.770833,2090.199951,2121.699951,2071.850098,2086.550049
...,...,...,...,...,...
28-04-23,19474.770833,2312.300049,2369.000000,2312.300049,2364.399902
02-05-23,19478.770833,2373.199951,2416.350098,2373.199951,2384.449951
03-05-23,19479.770833,2384.100098,2390.000000,2348.500000,2356.300049


In [47]:
def isSupport(df,i):
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
  and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
  and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2]

  return resistance

In [48]:
levels = []
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    levels.append((i,df['Low'][i]))
  elif isResistance(df,i):
    levels.append((i,df['High'][i]))
levels

In [49]:
def plot_all():
  fig, ax = plt.subplots()

  candlestick_ohlc(ax,df.values,width=0.6, \
                   colorup='green', colordown='red', alpha=0.8)

  date_format = mpl_dates.DateFormatter('%d %b %Y')
  ax.xaxis.set_major_formatter(date_format)
  fig.autofmt_xdate()

  fig.tight_layout()

  for level in levels:
    plt.hlines(level[1],xmin=df['Date'][level[0]],\
               xmax=max(df['Date']),colors='blue')
  fig.show()

In [50]:
plot_all()

In [51]:
s =  np.mean(df['High'] - df['Low'])

In [52]:
def isFarFromLevel(l):
  return np.sum([abs(l-x) < s  for x in levels]) == 0

In [53]:
levels = []
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    l = df['Low'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

  elif isResistance(df,i):
    l = df['High'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

In [54]:
levels = [[round(x, 2) for x in tup] for tup in levels]
second_values = []
for inner_list in levels:
    second_value = inner_list[1]
    second_values.append(second_value)

print(second_values)

[2058.0, 2297.65, 2199.0, 2107.0, 2246.25, 2416.35]


In [55]:
plot_all()